In [1]:
import json
import boto3
from faker import Faker
from datetime import datetime, timedelta
import uuid

In [2]:
# Initialize Faker instance
fake = Faker()

# AWS S3 setup
s3 = boto3.client('s3')

# Define your S3 bucket name
BUCKET_NAME = 'pismo-development-raw-events'

In [3]:
# Function to generate a single event
def generate_event():
    domain = fake.random_element(elements=('account', 'transaction'))
    event_type = fake.random_element(elements=('status-change', 'payment', 'creation'))
    
    event = {
        "event_id": str(uuid.uuid4()),  # Unique event ID
        "timestamp": datetime.now().isoformat(),  # Current timestamp
        "domain": domain,
        "event_type": event_type,
        "data": {
            "id": fake.random_int(min=100000, max=999999),  # Unique ID for account/transaction
            "old_status": fake.random_element(elements=('SUSPENDED', 'ACTIVE')),
            "new_status": fake.random_element(elements=('ACTIVE', 'INACTIVE')),
            "reason": fake.sentence()
        }
    }
    return event

In [4]:
print(generate_event())

{'event_id': '81ce37c9-2b11-4adf-b5c8-47d4c8f4cbb0', 'timestamp': '2024-10-03T09:29:10.864618', 'domain': 'account', 'event_type': 'status-change', 'data': {'id': 698828, 'old_status': 'SUSPENDED', 'new_status': 'ACTIVE', 'reason': 'National authority hot fight.'}}


In [5]:
# Function to upload events to S3
def upload_event_to_s3(event_data, file_name):
    s3.put_object(
        Bucket=BUCKET_NAME,
        Key=file_name,  # Unique file name for each event
        Body=json.dumps(event_data),
        ContentType='application/json'
    )
    print(f"Uploaded event to S3: {file_name}")